In [1]:
from datasets import load_dataset, DatasetDict
import pandas as pd

from collections import defaultdict

# Chargement des données

Les données proviennent du jeu "xtreme" dont sont extraites trois sous-ensembles correspondant aux langues parlées en Belgique, l'anglais en plus.

In [2]:
langs = ["nl", "fr", "de", "en"]
panx_be = defaultdict(DatasetDict)

for lang in langs:
    ds = load_dataset(path="xtreme", name=f"PAN-X.{lang}")
    panx_be[lang] |= ds

pd.DataFrame(
    {lang: [panx_be[lang]["train"].num_rows] for lang in langs },
    index=["number of training example"]
)


,nl,fr,de,en
number of training example,20000,20000,20000,20000


In [3]:
element = panx_be["fr"]["train"][5]
for key, value in element.items():
    print(f"{key}: {value}")

tokens: ['Comté', 'de', 'Motley']
ner_tags: [5, 6, 6]
langs: ['fr', 'fr', 'fr']


In [4]:
tags = panx_be["en"]["train"].features["ner_tags"].feature
def create_tag_names(batch) -> dict[str, list]:
    return {
        "ner_tags_str": [tags.int2str(n) for n in batch["ner_tags"]]
    }

for lang in langs:
    panx_be[lang] = panx_be[lang].map(create_tag_names, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [5]:
element = panx_be["en"]["train"][0]
for k, v in element.items():
    print(f"{k}: {v}")

tokens: ['R.H.', 'Saunders', '(', 'St.', 'Lawrence', 'River', ')', '(', '968', 'MW', ')']
ner_tags: [3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0]
langs: ['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en']
ner_tags_str: ['B-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O']


In [ ]:
{e: [l.count(e)] for e in unique}

In [36]:
d = defaultdict(list)
idx = list()

for lang in langs:
    for split in ["train", "validation", "test"]:
        l = [a for i in panx_be[lang][split]["ner_tags_str"] for a in i if a.startswith("B-")]
        unique = list(dict.fromkeys(l))
        d_tmp = {e: l.count(e) for e in unique}
        for key, value in d_tmp.items():
            d[key].append(value)
        idx.append(f"{lang} - {split}")
pd.DataFrame(d, idx)

,B-ORG,B-LOC,B-PER
nl - train,7778,9964,9308
nl - validation,3943,4835,4754
nl - test,3908,5133,4684
fr - train,7764,9718,8965
fr - validation,3910,4840,4406
fr - test,3885,4985,4499
de - train,8575,9778,9290
de - validation,4281,4968,4569
de - test,4157,4961,4750
en - train,9422,9345,9164
